In [1]:
%matplotlib inline
import pymc3 as pm
import theano.tensor as T
import theano
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler
import statsmodels.api as sm

import pickle

pd.options.display.max_columns=150
pd.options.display.max_rows=150

/Users/samuelholden/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv('banking_sector_weekly.csv')
df.dropna(subset=['vol', 'eps','weekly_return', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta'],inplace=True)
df['beg_date'] = pd.to_datetime(df['beg_date'])

df['ML3_return'] = (df['prior_wk_ret1']+df['prior_wk_ret2']+df['prior_wk_ret3'])/3.0
df['ML5_return'] = (df['prior_wk_ret1']+df['prior_wk_ret2']+df['prior_wk_ret3']+df['prior_wk_ret4']+df['prior_wk_ret5'])/5.0

scaler = StandardScaler()
df[['vol', 'eps', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta']] = scaler.fit_transform(df[['vol', 'eps', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta']])

df

,Unnamed: 0,Unnamed: 0.1,beg_date,end_date,conm,tic,announcement(maybe div),vol,eps,nearest_month,open_week,close_week,weekly_return,prior_wk_ret1,prior_wk_ret2,prior_wk_ret3,prior_wk_ret4,prior_wk_ret5,prior_wk_ret6,prior_wk_ret7,imports,exports,payroll,unemployment,unemployment_delta,ML3_return,ML5_return
7,7,146,2004-10-19,2004-10-24,JPMORGAN CHASE & CO,JPM,NaN,-0.045373,-0.002965,2004-10-01,38.75,37.47,-0.033032,-0.200339,-0.076999,0.509662,0.062079,0.109406,-0.076328,0.295346,0.913514,-0.821201,0.102768,-0.586339,-0.546234,0.007154,0.007339
8,8,147,2004-10-26,2004-10-31,JPMORGAN CHASE & CO,JPM,NaN,-0.154417,-0.135129,2004-10-01,37.02,38.60,0.042680,-0.662112,-0.199918,-0.077701,0.509775,0.061999,0.110638,-0.075926,0.913514,-0.821201,0.102768,-0.586339,-0.546234,-0.014077,-0.001048
9,9,148,2004-11-02,2004-11-07,JPMORGAN CHASE & CO,JPM,NaN,-0.125731,-0.135129,2004-11-01,38.50,39.35,0.022078,0.731135,-0.661732,-0.200609,-0.077557,0.509632,0.063223,0.111048,0.245792,0.370431,0.284133,-0.586339,0.034175,0.000570,0.006222
10,10,149,2004-11-09,2004-11-14,JPMORGAN CHASE & CO,JPM,NaN,-0.203807,-0.135129,2004-11-01,39.33,39.17,-0.004068,0.352023,0.731637,-0.662380,-0.200459,-0.077618,0.510926,0.063631,0.245792,0.370431,0.284133,-0.586339,0.034175,0.010575,0.004505
11,11,150,2004-11-16,2004-11-21,JPMORGAN CHASE & CO,JPM,NaN,-0.059867,-0.135129,2004-11-01,38.50,37.42,-0.028052,-0.129116,0.352492,0.730861,-0.662206,-0.200503,-0.076415,0.511351,0.245792,0.370431,0.284133,-0.586339,0.034175,0.020230,0.003944
12,12,151,2004-11-23,2004-11-28,JPMORGAN CHASE & CO,JPM,NaN,-0.239529,-0.135129,2004-11-01,37.65,37.69,0.001062,-0.570465,-0.128689,0.351751,0.730962,-0.662186,-0.199319,-0.076013,0.245792,0.370431,0.284133,-0.586339,0.034175,-0.003347,-0.000079
13,13,152,2004-11-30,2004-12-05,JPMORGAN CHASE & CO,JPM,NaN,-0.146264,-0.135129,2004-11-01,37.18,38.06,0.023669,-0.034704,-0.570076,-0.129386,0.351872,0.730789,-0.661073,-0.198922,0.245792,0.370431,0.284133,-0.586339,0.034175,-0.010353,0.006740
14,14,153,2004-12-07,2004-12-12,JPMORGAN CHASE & CO,JPM,NaN,-0.171927,-0.135129,2004-12-01,38.20,37.63,-0.014921,0.381295,-0.034268,-0.570733,-0.129240,0.351751,0.732117,-0.660695,0.980286,0.635238,1.060209,-0.532675,0.614584,-0.001107,0.002938
15,15,154,2004-12-14,2004-12-19,JPMORGAN CHASE & CO,JPM,NaN,-0.098973,-0.135129,2004-12-01,38.32,38.52,0.005219,-0.328838,0.381767,-0.034974,-0.570564,-0.129294,0.353020,0.732550,0.980286,0.635238,1.060209,-0.532675,0.614584,0.003270,-0.004462
16,16,155,2004-12-21,2004-12-26,JPMORGAN CHASE & CO,JPM,NaN,-0.194435,-0.135129,2004-12-01,38.65,39.16,0.013195,0.041789,-0.328428,0.381023,-0.034833,-0.570557,-0.128099,0.353439,0.980286,0.635238,1.060209,-0.532675,0.614584,0.004655,-0.002605


#### Let's first try fully pooled to see what's up

In [3]:
train_df = df[(df['beg_date']>pd.to_datetime('2014-01-01'))&(df['beg_date']<=pd.to_datetime('2017-01-01'))]
test_df = df[(df['beg_date']>=pd.to_datetime('2017-01-01'))]

x = train_df[['eps', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta','ML3_return','ML5_return']]

x = sm.add_constant(x)

y = train_df[['weekly_return']]

lm = sm.OLS(y,x)
results = lm.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          weekly_return   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     7.932
Date:                Mon, 18 Mar 2019   Prob (F-statistic):           1.01e-15
Time:                        12:33:04   Log-Likelihood:                 5036.6
No. Observations:                2319   AIC:                        -1.005e+04
Df Residuals:                    2305   BIC:                            -9965.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0004      0.002      0.222      0.824      -0.003       0.004
eps                -9.577e-06      0.001     -0.014      0.989      -0.001       0.001
prior_wk_ret1         -0.0015      0.001     -1.364      0.173      -0.004       0.001
prior_wk_ret2      -5.064e-05      0.001     -0.045      0.964      -0.002       0.002
prior_wk_ret3         -0.0046      0.001     -4.129      0.000      -0.007      -0.002
prior_wk_ret4          0.0015      0.001      1.318      0.188      -0.001       0.004
prior_wk_ret5          0.0042      0.001      3.675      0.000       0.002       0.006
prior_wk_ret6          0.0011      0.001      0.999      0.318      -0.001       0.003
prior_wk_ret7         -0.0023      0.001     -2.074      0.038      -0.005      -0.000
imports               -0.0048      0.002     -2.712      0.007      -0.008      -0.001
exports                0.0027      0.001      1.968      0.049    9.39e-06       0.005
payroll               -0.0071      0.002     -3.661      0.000      -0.011      -0.003
unemployment          -0.0105      0.002     -5.746      0.000      -0.014      -0.007
unemployment_delta    -0.0032      0.001     -4.270      0.000      -0.005      -0.002
ML3_return            -0.0001   3.62e-05     -3.087      0.002      -0.000   -4.08e-05
ML5_return         -4.775e-06   2.84e-05     -0.168      0.866   -6.04e-05    5.08e-05
==============================================================================
Omnibus:                      173.333   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              570.743
Skew:                           0.340   Prob(JB):                    1.16e-124
Kurtosis:                       5.334   Cond. No.                     4.16e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.28e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [4]:
train_df['pred'] = results.predict(x)

train_df.to_csv('ols_trainset_results.csv',index=False)

/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
x = test_df[['eps', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta','ML3_return','ML5_return']]

x = sm.add_constant(x)

test_df['pred'] = results.predict(x)

test_df.to_csv('ols_testset_results.csv',index=False)

/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Generalisation results are NAT good. Let us try partial pooling

In [6]:
le = LabelEncoder()

df['conm_code'] = le.fit_transform(df.tic)
df['weekly_return'] = df['weekly_return'].astype(theano.config.floatX)


train_df = df[(df['beg_date']>pd.to_datetime('2014-01-01'))&(df['beg_date']<=pd.to_datetime('2017-01-01'))]
test_df = df[(df['beg_date']>=pd.to_datetime('2017-01-01'))]

print(train_df.shape)
print(test_df.shape)

relevant_vars=['eps', 'prior_wk_ret1', 'prior_wk_ret2',
       'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
       'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
       'unemployment_delta','ML3_return','ML5_return']

conm_idx = train_df.conm_code.values
n_companies = len(df.conm.unique())

(2319, 28)
(659, 28)


In [7]:
conm_idx = theano.shared(np.array(train_df.conm_code.values,dtype='int32'))

x1_shared = theano.shared(np.array(train_df['prior_wk_ret1'],dtype='float32'))
x2_shared = theano.shared(np.array(train_df['prior_wk_ret2'],dtype='float32'))
x3_shared = theano.shared(np.array(train_df['prior_wk_ret3'],dtype='float32'))
x4_shared = theano.shared(np.array(train_df['prior_wk_ret4'],dtype='float32'))
x5_shared = theano.shared(np.array(train_df['prior_wk_ret5'],dtype='float32'))
x6_shared = theano.shared(np.array(train_df['prior_wk_ret6'],dtype='float32'))
x7_shared = theano.shared(np.array(train_df['prior_wk_ret7'],dtype='float32'))
x8_shared = theano.shared(np.array(train_df['eps'],dtype='float32'))
x9_shared = theano.shared(np.array(train_df['imports'],dtype='float32'))
x10_shared = theano.shared(np.array(train_df['exports'],dtype='float32'))
x11_shared = theano.shared(np.array(train_df['unemployment'],dtype='float32'))
x12_shared = theano.shared(np.array(train_df['unemployment_delta'],dtype='float32'))
x13_shared = theano.shared(np.array(train_df['ML3_return'],dtype='float32'))
x14_shared = theano.shared(np.array(train_df['ML5_return'],dtype='float32'))
#ann_output = theano.shared(train_df['weekly_return'])

In [31]:
with pm.Model() as hierarchical_model:
    # Hyperpriors for group nodes
    mu_a = pm.Normal('mu_a', mu=0., sd=100000**2)
    sigma_a = pm.HalfCauchy('sigma_a', 50)
    mu_b = pm.Normal('mu_b', mu=0., sd=100000**2)
    sigma_b = pm.HalfCauchy('sigma_b', 50)
    mu_b1 = pm.Normal('mu_b1', mu=0., sd=100000**2)
    sigma_b1 = pm.HalfCauchy('sigma_b1', 50)
    mu_b2 = pm.Normal('mu_b2', mu=0., sd=100000**2)
    sigma_b2 = pm.HalfCauchy('sigma_b2', 50)
    mu_b3 = pm.Normal('mu_b3', mu=0., sd=100000**2)
    sigma_b3 = pm.HalfCauchy('sigma_b3', 50)
    mu_b4 = pm.Normal('mu_b4', mu=0., sd=100000**2)
    sigma_b4 = pm.HalfCauchy('sigma_b4', 50)
    mu_b5 = pm.Normal('mu_b5', mu=0., sd=100000**2)
    sigma_b5 = pm.HalfCauchy('sigma_b5', 50)
    mu_b6 = pm.Normal('mu_b6', mu=0., sd=100000**2)
    sigma_b6 = pm.HalfCauchy('sigma_b6', 50)
    mu_b7 = pm.Normal('mu_b7', mu=0., sd=100000**2)
    sigma_b7 = pm.HalfCauchy('sigma_b7', 50)
    mu_b8 = pm.Normal('mu_b8', mu=0., sd=100000**2)
    sigma_b8 = pm.HalfCauchy('sigma_b8', 50)
    mu_b9 = pm.Normal('mu_b9', mu=0., sd=100000**2)
    sigma_b9 = pm.HalfCauchy('sigma_b9', 50)
    mu_b10 = pm.Normal('mu_b10', mu=0., sd=100000**2)
    sigma_b10 = pm.HalfCauchy('sigma_b10', 50)
    mu_b11 = pm.Normal('mu_b11', mu=0., sd=100000**2)
    sigma_b11 = pm.HalfCauchy('sigma_b11', 50)
    mu_b12 = pm.Normal('mu_b12', mu=0., sd=100000**2)
    sigma_b12 = pm.HalfCauchy('sigma_b12', 50)
    mu_b13 = pm.Normal('mu_b13', mu=0., sd=100000**2)
    sigma_b13 = pm.HalfCauchy('sigma_b13', 50)
    #mu_b14 = pm.Normal('mu_b14', mu=0., sd=100**2)
    #sigma_b14 = pm.HalfCauchy('sigma_b14', 5)
    

    # Intercept for each conm, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_companies).
    a = pm.Normal('a', mu=mu_a, sd=sigma_a, shape=n_companies)
    # beta 1 through 6
    b = pm.Normal('b', mu=mu_b, sd=sigma_b, shape=n_companies)
    b1 = pm.Normal('b1', mu=mu_b1, sd=sigma_b1, shape=n_companies)
    b2 = pm.Normal('b2', mu=mu_b2, sd=sigma_b2, shape=n_companies)
    b3 = pm.Normal('b3', mu=mu_b3, sd=sigma_b3, shape=n_companies)
    b4 = pm.Normal('b4', mu=mu_b4, sd=sigma_b4, shape=n_companies)
    b5 = pm.Normal('b5', mu=mu_b5, sd=sigma_b5, shape=n_companies)
    b6 = pm.Normal('b6', mu=mu_b6, sd=sigma_b6, shape=n_companies)
    b7 = pm.Normal('b7', mu=mu_b7, sd=sigma_b7, shape=n_companies)
    b8 = pm.Normal('b8', mu=mu_b8, sd=sigma_b8, shape=n_companies)
    b9 = pm.Normal('b9', mu=mu_b9, sd=sigma_b9, shape=n_companies)
    b10 = pm.Normal('b10', mu=mu_b10, sd=sigma_b10, shape=n_companies)
    b11 = pm.Normal('b11', mu=mu_b11, sd=sigma_b11, shape=n_companies)
    b12 = pm.Normal('b12', mu=mu_b12, sd=sigma_b12, shape=n_companies)
    b13 = pm.Normal('b13', mu=mu_b13, sd=sigma_b13, shape=n_companies)
    #b14 = pm.Normal('b14', mu=mu_b14, sd=sigma_b14, shape=n_companies)

    # Model error
    #['eps', 'prior_wk_ret1', 'prior_wk_ret2',
    #   'prior_wk_ret3', 'prior_wk_ret4', 'prior_wk_ret5', 'prior_wk_ret6',
    #   'prior_wk_ret7', 'imports', 'exports', 'payroll', 'unemployment',
    #   'unemployment_delta','ML3_return','ML5_return']
    
    sigma = pm.Gamma("sigma", alpha=10, beta=1)
    
    #dir_est=a[conm_idx] + b[conm_idx] * x1_shared + b1[conm_idx] * x2_shared + \
    #         b2[conm_idx] * x3_shared + b3[conm_idx] * x4_shared + \
    #         b4[conm_idx] * x5_shared + b5[conm_idx] * x6_shared + \
    #        b6[conm_idx] * x7_shared + b7[conm_idx] * x8_shared + \
    #        b8[conm_idx] * x9_shared + b9[conm_idx] * x10_shared + \
    #        b10[conm_idx] * x11_shared + b11[conm_idx] * x12_shared + \
    #        b12[conm_idx] * x13_shared + b13[conm_idx] * x14_shared
    
    dir_est=a[conm_idx] + b[conm_idx] * x1_shared + b7[conm_idx] * x8_shared + \
            b8[conm_idx] * x9_shared + b9[conm_idx] * x10_shared + \
            b10[conm_idx] * x11_shared + b11[conm_idx] * x12_shared + \
            b12[conm_idx] * x13_shared + b13[conm_idx] * x14_shared


    # Data likelihood
    dir_like = pm.Normal('dir_like', mu=dir_est, sd=sigma, observed=train_df.weekly_return)
    #dir_like = pm.Logistic('dir_like', mu=dir_est, s=sigma, observed=data_train.weekly_direction)
    #dir_like = pm.Bernoulli('dir_like',p=dir_est, observed=data_train.weekly_direction)
    
    start = pm.find_MAP()

/Users/samuelholden/anaconda3/lib/python3.6/site-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = 5,875.7, ||grad|| = 2,875.6: : 5001it [00:10, 465.09it/s]                             



In [42]:
start['b13']

array([0.14302244, 0.13652006, 0.13333085, 0.12492195, 0.1381759 ,
       0.13727806, 0.14531108, 0.14092308, 0.13527777, 0.13377549,
       0.13530562, 0.14130567, 0.1347302 , 0.13974459, 0.13192337])

In [44]:
conm_idx = test_df.conm_code.values

preds = start['a'][conm_idx] + start['b'][conm_idx] * test_df['prior_wk_ret1'] + start['b7'][conm_idx] * test_df['eps'] + \
            start['b8'][conm_idx] * test_df['imports'] + start['b9'][conm_idx] * test_df['exports'] + \
            start['b10'][conm_idx] * test_df['unemployment'] + start['b11'][conm_idx] * test_df['unemployment_delta'] + \
            start['b12'][conm_idx] * test_df['ML3_return'] + start['b13'][conm_idx] * test_df['ML5_return']

test_df['preds'] = preds

test_df.to_csv('bhm_testset_results_MAP.csv',index=False)

/Users/samuelholden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
